In [ ]:
%autocall 2

In [ ]:
from importlib import reload

import numpy as np

import time, pprint

import librosa, xmltodict, io, audiotsm, sounddevice, soundfile

from skimage import exposure

from scipy import ndimage

import glob, csv

import os

import gc

In [ ]:
import nptdms, tempfile

In [ ]:
from ipywidgets import interact, interactive, FloatSlider, IntSlider, Layout, HBox, VBox

from ipywidgets import Audio as audiowidget

from IPython.display import Audio

In [ ]:
%matplotlib ipympl

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

from matplotlib.widgets import Cursor, MultiCursor

In [ ]:
print(mpl.get_backend())
print(plt.isinteractive())
print(mpl.is_interactive())

mpl.interactive(False)
plt.ioff()

print(mpl.is_interactive())
print(plt.isinteractive())

mpl.rcParams['figure.figsize']

In [ ]:
import matplotlib.style as mplstyle
print(mplstyle.available)

# mplstyle.use('classic')
mplstyle.use('fast')

mpl.rcParams['path.simplify'] = True

mpl.rcParams['path.simplify_threshold'] = 1.0

mpl.rcParams['agg.path.chunksize'] = 10000

In [ ]:
import sys

mylibrarypaths = ["C:/Users/m142519/python", "H:/python"]

for p in mylibrarypaths:
    if os.path.isdir(p):
        if p not in sys.path:
            sys.path.append(p)

pprint.pprint(sys.path)

In [ ]:
# import ipython_memory_usage

In [ ]:
%load_ext ipython_memory_usage

In [ ]:
%imu_start

In [ ]:
import pyMRAW as photron
photron = reload(photron)

In [ ]:
pwd

In [ ]:
cd "C:/Users/m142519/Desktop/JN_SOVTE/JN_AnalogDataRecordings/2023-09-21/"

In [ ]:
cd "../Stef_Tee/"

In [ ]:
ls

In [ ]:
alltdmsfiles = sorted(glob.glob('*.tdms'))

In [ ]:
tdmsfilename = "SOVTEdata20230921-140507.tdms"

print("Stef's recordings:")
tdmsfilename = "SOVTEdata20231030-110829.tdms"

tdmsfile = nptdms.TdmsFile(tdmsfilename, memmap_dir = tempfile.gettempdir())

In [ ]:
tdmsfile.__dict__

In [ ]:
tdmsfile.properties

In [ ]:
pprint.pprint(tdmsfile.properties['Comments'])

In [ ]:
tdmsfile.groups()

In [ ]:
SOVTEdata = tdmsfile['SOVTEdata']

In [ ]:
SOVTEdata.channels()

In [ ]:
egga = SOVTEdata['EGG_A']
eggb = SOVTEdata['EGG_B']

egglt = SOVTEdata['EGG_LT']
egglt.data.dtype

In [ ]:
pmouth = SOVTEdata['micprobe_mouth']
pout = SOVTEdata['mic_outside']

In [ ]:
egglt.properties

In [ ]:
egglt.time_track(accuracy = 'ms')

In [ ]:
fsanalog = int(1 / egglt.properties['wf_increment'])
print("analog data sampling frequency: ", fsanalog)

In [ ]:
audio = Audio(egga.data, rate = fsanalog)

In [ ]:
audio

In [ ]:
plt.close('all')

gc.collect()

In [ ]:
fmax = 600

Tmin = 1 / fmax

print(int(Tmin * fsanalog))

pmic = pout

peakloc, mad = find_peaks_fast(pmic.data, neighbors = int(Tmin * fsanalog))

f0peaks_index = peakloc[:-1]
f0peaks = 1 / (pmic.time_track()[peakloc[1:]] - pmic.time_track()[peakloc[:-1]])

In [ ]:
f0, voicedflag, voicedprobs = librosa.pyin(pout.data, fmin = 100, fmax = 600, sr = fsanalog, hop_length = 2048 // 8, frame_length = 2048)

In [ ]:
f0.shape

In [ ]:
f0times = librosa.times_like(f0, sr = fsanalog, hop_length = 2048 // 8)

In [ ]:
f0samples = librosa.samples_like(f0, hop_length = 2048 // 8)

In [ ]:
try:
    ax.clear()
    fig.clear()
    plt.close(fig = fig)
    del fig, ax
except Exception as e:
    print(e)
    plt.close('all')

gc.collect()

plt.ioff()
# plt.ion()

fig = plt.figure(figsize = (18, 13), clear = True)
fig.canvas.header_visible = False

ax, ax1 = fig.subplots(nrows = 2, ncols = 1, sharex = True, height_ratios = [0.49] * 2)

t0 = 50 * 3000
t1 = 50 * 5000

if False:
    ax.plot((egga.time_track() * 1000), egga.data, 'r-', label = 'EGG_A')

    ax.plot(smootha[:, 0] * 1000, smootha[:, 1], 'g.-', label = 'LOWESS EGG_A')

    ax1.plot(egga.time_track() * 1000, egga.data - smootha[:, 1], 'r-', label = 'EGG_A - LOWESS EGG_A')

if False:
    ax.plot(egglt.time_track() * 1000, egglt.data, 'b-', label = 'EGG_LT', zorder = 100)
    
    ax.plot(smoothLT[:, 0] * 1000, smoothLT[:, 1], 'k.-', label = 'LOWESS EGG_LT')

    ax1.plot(egglt.time_track() * 1000, egglt.data - smoothLT[:, 1], 'b-', label = 'EGG_LT - LOWESS EGG_LT')

if True:
    pmic = pmouth
    pmic = pout
    
    ax.plot(# pmic.time_track() * 1000, 
            pmic.data, 'b-', label = 'mic')
    
    ax.plot(peakloc, # pmic.time_track()[peakloc] * 1000, 
            pmic.data[peakloc], 'ro', label = 'peaks')

    ax1.plot(peakloc[:-1], # pmic.time_track()[peakloc[:-1]] * 1000,
             1 / (pmic.time_track()[peakloc[1:]] - pmic.time_track()[peakloc[:-1]]), 'b.', label = 'pitch')

    # ax1.plot(f0times * 1000, f0, 'r.', label = 'pyin')

    ax1.scatter(f0samples, # f0times * 1000, 
                f0, s = voicedprobs * 50, color = 'red', marker = '.', label = 'pyin', zorder = 100)

# ax1.set_xlabel('time [milliseconds]')

ax.grid(which = 'both')
ax1.grid(which = 'both')

ax.legend()
ax1.legend()

# multicursor = MultiCursor(canvas = fig.canvas, axes = [ax, ax1], useblit = True, horizOn = False, vertOn = True, color = 'red', linewidth = 1)

# ax.set_ylim(bottom = pmouth.data.min(), top = pmouth.data.max())


In [ ]:
display(fig.canvas)

In [ ]:
x0, x1 = ax1.get_xlim()

print(int(x0), int(x1))

In [ ]:
f0peaks_indexselected = (np.array(f0peaks_index) > int(x0)) & (np.array(f0peaks_index) < int(x1))

f0peaks_selected = np.ma.masked_array(f0peaks, mask = ~f0peaks_indexselected)
f0peaks_selected.fill_value = np.nan

f0peaks_selected.compressed()

In [ ]:
f0samples_selected = (f0samples > int(x0)) & (f0samples < int(x1))

f0_selected = np.ma.masked_invalid(f0)
f0_selected.fill_value = np.nan

f0_selected.mask = np.logical_or(f0_selected.mask, ~f0samples_selected)

# f0_selected.compressed()

In [ ]:
np.savez_compressed("C:/Users/m142519/Desktop/Jupyter Notebooks/selectedf0s.npz", f0_selected = f0_selected.compressed(), f0peaks_selected = f0peaks_selected.compressed())

In [ ]:
smoothegga = egga.data - smootha[:, 1]

In [ ]:
opened = smoothegga[int(x0):int(x1)]

In [ ]:
closed = smoothegga[int(x0):int(x1)]

In [ ]:
openglide = smoothegga[int(x0):int(x1)]

In [ ]:
closedglide = smoothegga[int(x0):int(x1)]

In [ ]:
peakclosedglide, mad = find_peaks_fast(closedglide, neighbors = int(Tmin * fsanalog))

In [ ]:
eggperiods = [-closedglide[p1:p2] for p1, p2 in zip(peakclosedglide[:-1], peakclosedglide[1:])]

In [ ]:
[(e - e.min()) / (e.max() - e.min()) for e in eggperiods]

In [ ]:
mpl.collections.QuadMesh(facecolors = , edgecolors = 'face')

In [ ]:
try:
    figegg.clear()
    axegg.clear()
    plt.close(fig = figegg)
    del figph, axegg
except Exception as e:
    print("Exception: ", e)

figegg, axegg = plt.subplots(figsize = (15, 8))
figegg.canvas.header_visible = False

axegg.plot(-openglide, 'b-', label = 'opened')
axegg.plot(-closedglide, 'r-', label = 'closed')

axegg.plot(peakclosedglide, -closedglide[peakclosedglide], 'ko')

axegg.grid(which = 'both')
axegg.legend()

figegg.canvas.draw()

In [ ]:
display(figegg.canvas)

In [ ]:
try:
    fig2d.clear()
    ax2d.clear()
    ax2d.clear()
    plt.close(fig = fig2d)
    del fig2d, ax2d
except Exception as e:
    print("Exception: ", e)

fig2d, ax2d = plt.subplots(figsize = (15, 8))
fig2d.canvas.header_visible = False

wclosedglide = np.reshape(np.diff(-closedglide[:180001]), (-1, 1000)).T

ax2d.imshow((wclosedglide - wclosedglide.min(axis = 0)) / (wclosedglide.max(axis = 0) - wclosedglide.min(axis = 0)), aspect = 'auto', origin = 'lower')

fig2d.colorbar(ax2d.images[0])

fig2d.canvas.draw()

In [ ]:
display(fig2d.canvas)

In [ ]:
try:
    figph.clear()
    axph1.clear()
    axph2.clear()
    plt.close(fig = figph)
    del figph, axph1, axph2
except Exception as e:
    print("Exception: ", e)

figph, [axph1, axph2] = plt.subplots(nrows = 2, figsize = (15, 12), sharex = True, sharey = True)
figph.canvas.header_visible = False

axph1.plot(-openglide[:-1], np.diff(-openglide), 'b-', label = 'opened')
axph2.plot(-closedglide[:-1], np.diff(-closedglide), 'r-', label = 'closed')

axph1.grid(which = 'both')
axph2.grid(which = 'both')

axph1.legend()
axph2.legend()

figph.canvas.draw()

In [ ]:
display(figph.canvas)

In [ ]:
try:
    fighist.clear()
    axhist.clear()
    plt.close(fig = fighist)
    del fighist, axhist
except Exception as e:
    print("Exception: ", e)

fighist, axhist = plt.subplots(figsize = (15, 8))

axhist.hist(f0[:3500], bins = 320 // 2, range = (180, 500))

axhist.grid(which = 'both')

axhist.set_xlabel('pitch frequency [Hz]')

fighist.canvas.draw()

In [ ]:
display(fighist.canvas)

In [ ]:
import statsmodels.api as sm

In [ ]:
egglt.time_track().shape

In [ ]:
from scipy import signal

In [ ]:
sosbandpass = signal.iirfilter(N = 7, Wn = [80, 10000], btype = 'bandpass', ftype = 'butter', output = 'sos', analog = False, fs = fsanalog)

w, h = signal.sosfreqz(sosbandpass, worN = 8 * 1024, fs = fsanalog)

plt.close('all')

plt.semilogx(w, 20 * np.log10(abs(h)))

plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude [dB]')

plt.margins(0, 0.1)

plt.grid(which = 'both', axis = 'both')

plt.axvline(80, color = 'green') # cutoff frequency
plt.axvline(10000, color = 'green')

plt.gca().set_xlim(left = 60, right = 15000)

plt.gca().set_ylim(bottom = -10, top = 3)


In [ ]:
display(plt.gcf().canvas)

In [ ]:
sosbandpass = signal.iirfilter(N = 7, Wn = [80, 10000], btype = 'bandpass', ftype = 'butter', output = 'sos', analog = False, fs = fsanalog)

bandpass_egga = signal.sosfiltfilt(sosbandpass, egga)

bandpass_egga = bandpass_egga[:3204037]

bandpass_egga /= np.abs(bandpass_egga).max()

In [ ]:
fmax = 600

Tmin = 1 / fmax

p, _ = signal.find_peaks(bandpass_egga, distance = int(Tmin * fsanalog))

f0p_index = p[:-1]

f0p = 1 / (egga.time_track()[p[1:]] - egga.time_track()[p[:-1]])

In [ ]:
peakloc, _ = find_peaks_fast(bandpass_egga, neighbors = int(Tmin * fsanalog))

f0peaks_index = peakloc[:-1]

f0peaks = 1 / (egga.time_track()[peakloc[1:]] - egga.time_track()[peakloc[:-1]])

In [ ]:
bandpassaudio = Audio(bandpass_egga, rate = fsanalog)

In [ ]:
audio

In [ ]:
bandpassaudio

In [ ]:
gc.collect()

In [ ]:
lowess_egga, _ = soundfile.read(tdmsfilename.replace('.tdms', '-LOWESS_EGG_A.wav'))

lowess_egga = lowess_egga[:3204037]

lowess_egga /= np.abs(lowess_egga).max()

In [ ]:
sosbandpass = signal.iirfilter(N = 7, Wn = [80, 10000], btype = 'bandpass', ftype = 'butter', output = 'sos', analog = False, fs = fsanalog)

bandpasslowess_egga = signal.sosfiltfilt(sosbandpass, lowess_egga)

bandpasslowess_egga /= np.abs(bandpasslowess_egga).max()

In [1]:
f0, voicedflag, voicedprobs = librosa.pyin(bandpasslowess_egga, fmin = 100, fmax = 600, sr = fsanalog, hop_length = 2048 // 16, frame_length = 2048, resolution = 0.1, )

NameError: name 'librosa' is not defined

In [ ]:
plt.close('all')

gc.collect()

# plt.plot(egga.time_track() * 1000, egga.data / np.abs(egga.data).max(), label = 'raw EGG_A', zorder = 100)

# plt.plot(bandpass_egga, label = 'bandpass filtered')

plt.plot(bandpasslowess_egga - bandpass_egga, label = 'lowess')

# plt.plot(egga.time_track() * 1000, bandpass_egga / np.abs(bandpass_egga).max(), label = 'bandpass EGG_A')

# plt.plot(egga.time_track() * 1000, bandpasslowess_egga / np.abs(bandpasslowess_egga).max(), label = 'bandpass lowess')

# plt.plot(egga.time_track()[p] * 1000, bandpass_egga[p] / np.abs(bandpass_egga).max(), 'r.', label = 'signal.find_peaks')

# plt.plot(egga.time_track()[peakloc] * 1000, bandpass_egga[peakloc] / np.abs(bandpass_egga).max(), 'ko', mfc = 'None', label = 'find_peaks_fast')

plt.legend()

plt.gcf().set_size_inches((15, 8))

plt.gcf().canvas.header_visible = False

In [ ]:
display(plt.gcf().canvas)

In [ ]:
plt.gca().get_xlim()

In [ ]:
# communicate with a running Audacity application

TONAME = '\\\\.\\pipe\\ToSrvPipe'
FROMNAME = '\\\\.\\pipe\\FromSrvPipe'
EOL = '\r\n\0'

try:
    TOFILE.close()
    del TOFILE
except:
    pass

try:
    FROMFILE.close()
    del FROMFILE
except:
    pass

print("Write to  \"" + TONAME + "\"", end = '')
if not os.path.exists(TONAME):
    print(" ... does not exist.  Ensure Audacity is running with mod-script-pipe.")
else:
    print()

print("Read from \"" + FROMNAME +"\"", end = '')
if not os.path.exists(FROMNAME):
    print(" ... does not exist.  Ensure Audacity is running with mod-script-pipe.")
else:
    print()

try:
    assert TOFILE
except:
    TOFILE = open(TONAME, 'w')
    print("-- Pipe to write to has been opened")
try:
    assert FROMFILE
except:
    FROMFILE = open(FROMNAME, 'rt')
    print("-- Pipe to read from has been opened")

def send_command(command, debug = False):
    """Send command."""
    if debug:
        print("Send: " + command)
    TOFILE.write(command + EOL)
    TOFILE.flush()

def get_response():
    """Return the command response."""
    result = ''
    line = ''
    while True:
        result += line
        line = FROMFILE.readline()
        if line == '\n' and len(result) > 0:
            break
    return result

def audacity(command, debug = False):
    """Send one command, and return the response."""
    send_command(command)
    response = get_response()
    if debug:
        print("Rcvd: " + response)
    return response

In [ ]:
audacity('NewMonoTrack')
audacity('SetTrack: Name="pitch"')
audacity('SetTrackStatus: Name="pitch" Selected=1 Focused=1')

In [ ]:
trackinfo = audacity('GetInfo: Type=Tracks Format=JSON')
trackinfo

In [ ]:
exec("dict = " + trackinfo.splitlines()[2])

In [ ]:
dict[0]

In [ ]:
f0steps = 5

for ind in range(600, 800, f0steps):

    audacity('Select: Track=3 Start=%f End=%f RelativeTo=ProjectStart' % (peakloc[ind] / fsanalog, peakloc[ind+f0steps] / fsanalog))

    audacity('Chirp: StartFreq=%f EndFreq=%f StartAmp=0.9 EndAmp=0.9' % (f0peaks[ind], f0peaks[ind+f0steps]))

In [ ]:
bandpassfilename = tdmsfilename.replace('.tdms', '-bandpass_EGG_A.wav')

soundfile.write(bandpassfilename, bandpass_egga / np.abs(bandpass_egga).max(), fsanalog)

In [ ]:
eggafilename = tdmsfilename.replace('.tdms', '-EGG_A.wav')

audacity('Import2: Filename=%s' % os.path.abspath(eggafilename))
audacity('FitInWindow')
audacity('SetTrack: Name="EGG_A"')

In [ ]:
audacity('Import2: Filename=%s' % os.path.abspath(bandpassfilename))
audacity('FitInWindow')
audacity('SetTrack: Name="bandpass_EGG_A"')

In [ ]:
lowess_eggafilename = tdmsfilename.replace('.tdms', '-LOWESS_EGG_A.wav')

audacity('Import2: Filename=%s' % os.path.abspath(lowess_eggafilename))
audacity('FitInWindow')
audacity('SetTrack: Name="lowess_EGG_A"')

In [ ]:
audacity('Select: Track=1')
audacity('SetTrack: Selected=1 Focused=1')

# audacity('Normalize: PeakLevel=0')

In [ ]:
diffsignal = bandpasslowess_egga

diffsignal /= np.abs(diffsignal).max()

soundfile.write('diffsignal.wav', diffsignal, fsanalog)

In [ ]:
audacity('Import2: Filename=%s' % os.path.abspath('diffsignal.wav'))
audacity('FitInWindow')
audacity('SetTrack: Name="diff"')

In [ ]:
dt = 1 / fsanalog
dt

In [ ]:
frac = 5 * 10000 / len(egglt.data)
print(frac)

smoothLT = sm.nonparametric.lowess(exog = egglt.time_track(), endog = egglt.data, frac = frac, is_sorted = True, delta = 5e-3, it = 1)

In [ ]:
channel = 'LOWESS_EGG_LT'

wavfilename = tdmsfilename.replace('.tdms', '-%s.wav' %channel)

print(wavfilename)

In [ ]:
soundfile.write(wavfilename, egglt.data - smoothLT[:, 1], fsanalog)

In [ ]:
frac = 1000 / len(egga.data)
print(frac)

smootha = sm.nonparametric.lowess(exog = egga.time_track(), endog = egga.data, frac = frac, is_sorted = True, delta = 1e-3, it = 1)

In [ ]:
channel = 'LOWESS_EGG_A'

wavfilename = tdmsfilename.replace('.tdms', '-%s.wav' %channel)

print(wavfilename)

In [ ]:
soundfile.write(wavfilename, egga.data - smootha[:, 1], fsanalog)

In [ ]:
frac = 1000 / len(eggb.data)
print(frac)

smoothb = sm.nonparametric.lowess(exog = eggb.time_track(), endog = eggb.data, frac = frac, is_sorted = True, delta = 1e-3, it = 1)

In [ ]:
channel = 'LOWESS_EGG_B'

wavfilename = tdmsfilename.replace('.tdms', '-%s.wav' %channel)

print(wavfilename)

In [ ]:
soundfile.write(wavfilename, eggb.data - smoothb[:, 1], fsanalog)

In [ ]:
gc.collect()

In [ ]:
cd "C:\Users\m142519\Desktop\JN_SOVTE\JN_Experiment01_Cross\HSV_20230818_133111"

In [ ]:
ls

In [ ]:
camerafile = "HSV_20230818_133111.cihx"

In [ ]:
cih = photron.get_cih(camerafile)

In [ ]:
f = open(camerafile, 'rb')

a = f.read()

f.close()

cihstart = a.find(b'<cih>')

cihend = a.find(b'</cih>') + 6

In [ ]:
cih = xmltodict.parse(a[cihstart:cihend], dict_constructor = dict)['cih']

In [ ]:
fileformat = cih['imageFileInfo']['fileFormat']
colorbit = int(cih['imageDataInfo']['colorInfo']['bit'])
effectivebitdepth = int(cih['imageDataInfo']['effectiveBit']['depth'])
ebs = cih['imageDataInfo']['effectiveBit']['side']
orgtotalframe = int(cih['frameInfo']['recordedFrame'])
totalframe = int(cih['frameInfo']['totalFrame'])

In [ ]:
print(fileformat,
      colorbit,
      effectivebitdepth,
      ebs,
      orgtotalframe,
      totalframe)

In [ ]:
rawimages = photron.load_images(''.join([os.path.splitext(camerafile)[0], '.mraw']), cih = cih)

In [ ]:
print()
print("shape: ", rawimages.shape)
print("dtype: ", rawimages.dtype)

print("rawimages: ", rawimages.__repr__())

try:
    height = cih["Image Height"]
    width = cih["Image Width"]
    Nframes = cih["Total Frame"]
    fps = cih["Record Rate(fps)"]
except:
    height = int(cih['imageDataInfo']['resolution']['height'])
    width = int(cih['imageDataInfo']['resolution']['width'])
    Nframes = int(cih['frameInfo']['totalFrame'])
    fps = int(cih['recordInfo']['recordRate'])

print()
print("height: ", height)
print("width: ", width)
print("Nframes: ", Nframes)
print("fps: ", fps)

In [ ]:
import pprint

In [ ]:
pprint.pprint(cih)

In [ ]:
import mediapy as media

In [ ]:
media.set_ffmpeg("C:\\Users\\m142519\\Programs\\ffmpeg-master-latest-win64-gpl\\bin\\ffmpeg.exe")

In [ ]:
if "C:/Users/m142519/Programs" not in sys.path:
    sys.path.append("C:/Users/m142519/Programs")

In [ ]:
def convertrawimage(rawimages, cih = cih, Nstart = 0, Nconvert = 1):
    r = rawimages

    try:
        height = np.int64(cih["Image Height"])
        width = np.int64(cih["Image Width"])
        Nframes = np.int64(cih["Total Frame"])
    except:
        height = np.int64(cih['imageDataInfo']['resolution']['height'])
        width = np.int64(cih['imageDataInfo']['resolution']['width'])
        Nframes = np.int64(cih['frameInfo']['totalFrame'])

    if Nstart < 0:
        Nstart = 0
    
    if Nstart >= Nframes:
        Nstart = Nframes - 1
        
    if Nstart + Nconvert > Nframes:
        Nconvert = Nframes - Nstart
    
    Na = np.int64(Nstart) * height * width // 2 * 3
    Nb = np.int64(Nstart + Nconvert) * height * width // 2 * 3

    t0 = time.perf_counter()
    byte01 = r[Na + 0:Nb:3].astype(np.uint16)
    t1 = time.perf_counter()
    byte02 = r[Na + 1:Nb:3].astype(np.uint16)
    t2 = time.perf_counter()
    byte03 = r[Na + 2:Nb:3].astype(np.uint16)
    t3 = time.perf_counter()

    # bitside = cih['EffectiveBit Side'].lower() # 'lower'

    # if bitside == 'lower':
    uint12_01 = np.left_shift(byte01, 4)
    t4 = time.perf_counter()
    uint12_01 += np.right_shift(byte02, 4)
    t5 = time.perf_counter()
    
    uint12_02 = byte03
    t6 = time.perf_counter()
    uint12_02 += np.left_shift(np.bitwise_and(byte02, 0x0F), 8)
    t7 = time.perf_counter()
    # else:
    #    raise NotImplementedError()

    frame = np.stack((uint12_01, uint12_02), axis = 1)
    t8 = time.perf_counter()
    
    timings = [t0, t1, t2, t3, t4, t5, t6, t7, t8]
    
    return frame.reshape((-1, height, width)), timings

In [ ]:
gc.collect()

In [ ]:
np.seterr(all = 'raise', over = 'raise', invalid = 'raise', under = 'raise')

In [ ]:
# demonstrate some error with overflow of long_scalars, np.int32

Nblock = 4000

for framenum in np.arange(16000, Nframes, Nblock, dtype = np.int32):
    Nstart = framenum
    Nconvert = Nblock
    Nb = np.int32(Nstart + Nconvert) * height * width // 2 * 3

In [ ]:
%%timeit Nstart = np.random.randint(0, Nframes)
image, timings = convertrawimage(rawimages, Nstart = Nstart, Nconvert = 1)

In [ ]:
# %%timeit -n1 -r1
# 1.59 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

triggertimems = 9041
# triggertimems = 0

starttimems, endtimems = [26500, 27800]

starttimems, endtimems = [32500, 33700]

# starttimems, endtimems = [54904, 55770]

# starttimems, endtimems = [56317, 56796]

Nstart = fps / 1000 * (starttimems - triggertimems)
Nend = fps / 1000 * (endtimems - triggertimems)
Nconvert = Nend - Nstart

print(starttimems, Nstart)
print(endtimems, Nend)
print(Nconvert)

highspeedvideo, _ = convertrawimage(rawimages, Nstart = Nstart, Nconvert = Nconvert)

In [ ]:
gc.collect()

In [ ]:
equalize_highspeed = exposure.equalize_adapthist(highspeedvideo, 
                                                 kernel_size = width / 10, 
                                                 clip_limit = 0.05,
                                                 nbins = 256)

In [ ]:
del highspeedvideo
gc.collect()

In [ ]:
o = exposure.adjust_gamma(equalize_highspeed, gamma = 1 / 2.2)

In [ ]:
del equalize_highspeed
gc.collect()

import tempfile

tempvideofilename = os.path.join(tempfile.gettempdir(), 'tempvideo.mp4')

with media.VideoWriter(tempvideofilename, (height, width), codec = 'h264', fps = 30, input_format = 'gray') as writer:
    for image in highspeedvideo:
        equalize_image = exposure.equalize_adapthist(image, 
                                                 kernel_size = width / 10, 
                                                 clip_limit = 0.05,
                                                 nbins = 256)
        writer.add_image(equalize_image)

media.show_video(media.read_video(tempvideofilename))

del image, writer

In [ ]:
pwd

In [ ]:
media.set_show_save_dir('C:/Users/m142519/Desktop/Jupyter Notebooks')

In [ ]:
media._config.show_save_dir

In [ ]:
videotitle = camerafile.replace('.cihx', '.%d-%d' % (Nstart, Nstart + Nconvert))

media.show_video(o, title = videotitle, codec = 'h264', fps = 10, height = 2 * height)

In [ ]:
media.show_video(media.read_video(os.path.join(media._config.show_save_dir, videotitle + '.mp4')))

In [ ]:
plt.close(fig = fig)
del fig
gc.collect()

In [ ]:
try:
    [ax.clear() for a in ax]
    fig.clear()
    plt.close(fig = fig)
    del ax
    del fig
except:
    pass

fig, ax = plt.subplots(ncols = 3, figsize = (17.3, 6.2), sharex = True, sharey = True)
fig.canvas.header_visible = False

plt.subplots_adjust(left = 0.05, bottom = 0.05, top = 0.95, hspace = 0.01, wspace = 0.01)

im1 = ax[0].imshow((o[0, :, :]), cmap = plt.cm.gray, aspect = 'auto', interpolation = 'none')
im1title = ax[0].set_title('framenr: %4d' % 0)

im2 = ax[1].imshow(ndimage.grey_opening(o[0, :, :], size = (3, 3)), cmap = plt.cm.gray, aspect = 'auto', interpolation = 'none')
im2title = ax[1].set_title('grey_opening')

im3 = ax[2].imshow(ndimage.grey_closing(o[0, :, :], size = (3, 3)), cmap = plt.cm.gray, aspect = 'auto', interpolation = 'none')
im3title = ax[2].set_title('grey_closing')

# im4 = ax[3].imshow(ndimage.morphological_gradient(o[0, :, :], size = (3, 3)), cmap = plt.cm.gray, aspect = 'auto', interpolation = 'none')
# im4title = ax[3].set_title('morphological_gradient')

# kymoline
xdata = []
ydata = []

# kymoline, = ax[0].plot(xdata, ydata, 'gx-', markersize = 10, linewidth = 2, zorder = 10, visible = True)
kymolinesG = [a.plot(xdata, ydata, 'gx-', markersize = 10, linewidth = 2, zorder = 10, visible = True)[0] for a in ax]
kymolinesB = [a.plot(xdata, ydata, 'bx-', markersize = 10, linewidth = 2, zorder = 10, visible = True)[0] for a in ax]

if 0:
    cursor1 = Cursor(ax1, horizOn = True, vertOn = True, useblit = True, color = 'red', linewidth = 1)
    cursor2 = Cursor(ax2, horizOn = True, vertOn = True, useblit = True, color = 'red', linewidth = 1)
    cursor3 = Cursor(ax3, horizOn = True, vertOn = True, useblit = True, color = 'red', linewidth = 1)

cursor = MultiCursor(fig.canvas, (ax[0], ax[1], ax[2]), color = 'red', lw = 1, horizOn = True, vertOn = True, useblit = True)

fig.canvas.draw_idle()

def onclick(event):
    if fig.canvas.widgetlock.locked():
        return
        
    if event.inaxes in ax:
        xdata.append(event.xdata)
        ydata.append(event.ydata)
        
        [k.set_xdata(xdata[-4:-2]) for k in kymolinesG]
        [k.set_ydata(ydata[-4:-2]) for k in kymolinesG]
        [k.set_xdata(xdata[-2:]) for k in kymolinesB]
        [k.set_ydata(ydata[-2:]) for k in kymolinesB]
        
    fig.canvas.draw_idle()

try:
    fig.canvas.mpl_disconnect(ka)
except:
    pass
ka = fig.canvas.mpl_connect('button_press_event', onclick)
        
def show_frame(framenr = 0):
    global o
    
    im1.set_data((o[framenr, :, :]))
    im1title.set_text('framenr: %4d' % framenr)
    
    im2.set_data(ndimage.grey_opening(o[framenr, :, :], size = (3, 3)))
    im3.set_data(ndimage.grey_closing(o[framenr, :, :], size = (3, 3)))
    # im4.set_data((ndimage.morphological_gradient(o[framenr, :, :], size = (3, 3))))
    
    fig.canvas.draw_idle()

framenrSlider = IntSlider(max = len(o) - 1, description = 'Frame', continuous_update = True, layout = Layout(width = '99%'))

w = interactive(show_frame, 
                framenr = framenrSlider)

display(fig.canvas)
display(w)


In [ ]:
fig.get_size_inches

In [ ]:
import matplotlib.animation as animation

In [ ]:
def animate(framenr):
    if 0:
        im.set_data(o[framenr, :, :])
        imtitle.set_text('framenr: %4d' % framenr)

        return im, imtitle
    
    framenrSlider.value = framenr
    return framenrSlider,

In [ ]:
anim = animation.FuncAnimation(fig, animate, frames = range(1100, 1400), interval = 100, blit = True, cache_frame_data = True)

In [ ]:
anim.pause()

del anim
gc.collect()

In [ ]:
kymotopG = int(np.rint(ydata[-4]))
kymobottomG = int(np.rint(ydata[-3]))

kymolineG = kymobottomG

kymoleftG = int(np.rint(xdata[-4]))
kymorightG = int(np.rint(xdata[-3]))
##################################################
kymotopB = int(np.rint(ydata[-2]))
kymobottomB = int(np.rint(ydata[-1]))

kymolineB = kymobottomB

kymoleftB = int(np.rint(xdata[-2]))
kymorightB = int(np.rint(xdata[-1]))

In [ ]:
def getkymo(movie, kymotop, kymobottom, kymoleft, kymoright):
    columns = np.linspace(kymoright, kymoleft, num = kymoright - kymoleft + 1).astype(int)
    
    rows = np.rint(np.linspace(kymobottom, kymotop, num = kymoright - kymoleft + 1)).astype(int)

    kymo = movie[:, rows, columns]
    return kymo

kymo1 = getkymo(o, kymotopG, kymobottomG, kymoleftG, kymorightG)
kymo2 = getkymo(o, kymotopB, kymobottomB, kymoleftB, kymorightB)

kymo1 = o[:, int(np.rint(ydata[-2])), kymoright:kymoleft:-1]

kymo2 = o[:, kymoline, kymoright:kymoleft:-1]

In [ ]:
kymo = np.hstack((kymo1, kymo2))
print(kymo.shape)

In [ ]:
pwd

In [ ]:
[hsvaudio, hsvtrigger], fsaudio = librosa.load('../hsv.WAV', sr = None, mono = False)

In [ ]:
starttimesamples = starttimems * fsaudio // 1000
endtimesamples = endtimems * fsaudio // 1000

audiokymo = hsvaudio[starttimesamples - fsaudio//2 * 0 : endtimesamples + fsaudio//2 * 0]

In [ ]:
f0max = 600 # maximum peak frequency in Hz

[peakloc, mad] = find_peaks_fast(audiokymo, neighbors = fsaudio // f0max)

In [ ]:
plt.close(fig = figk)
del figk
gc.collect()

In [ ]:
try:
    [a.clear() for a in axk]
    axaudio.clear()
    del axk, axaudio
    figk.clear()
    plt.close(fig = figk)
    del figk
    del cursoraudio
except:
    pass

figk, axk = plt.subplots(nrows = 4, figsize = (16.5, 8.5), height_ratios = [1, 1, 2, 2], sharex = True, sharey = False)
figk.canvas.header_visible = False
plt.subplots_adjust(left = 0.05, bottom = 0.07, top = 0.95, wspace = 0.05, hspace = 0.05)

# axaudio = axk.twinx()
axaudio = axk[0]

audioframetime = np.arange(0, endtimesamples - starttimesamples) / fsaudio * fps

audiodelayfromglottis = 0.17 / 344 * fps # in highspeed frames

audiokymotime = audioframetime + audiodelayfromglottis

axaudio.plot(audiokymotime, audiokymo, 'r-', zorder = 10, alpha = 1.0)

axaudio.plot(audiokymotime[peakloc], audiokymo[peakloc], 'go')
axaudio.grid(visible = True, which = 'both')

axaudio.set_ylim(bottom = audiokymo.min() * 1.1, top = audiokymo.max() * 1.1) # minimum should be negative

axfrequ = axk[1]
localfrequ = fsaudio / np.diff(peakloc)
axfrequ.plot(audiokymotime[peakloc[1:]], localfrequ, 'bo-')
axfrequ.set_ylim(bottom = localfrequ.min() * 0.9, top = localfrequ.max() * 1.1)
axfrequ.grid(visible = True, which = 'both')

axfrequ.set_ylabel('local frequency [Hz]')

# imk = axk.imshow(ndimage.grey_opening(kymo.T, size = (3, 3)), cmap = plt.cm.gray, aspect = 'auto')
imk1 = axk[2].imshow(ndimage.grey_opening(kymo1.T, size = (3, 3)), cmap = plt.cm.gray, aspect = 'auto')
imk2 = axk[3].imshow(ndimage.grey_opening(kymo2.T, size = (3, 3)), cmap = plt.cm.gray, aspect = 'auto')

axk[3].set_xlabel(f'time [frames wrt {fps} fps]')
axk[3].set_ylabel('right vocal fold at top, left vocal fold at bottom')

# axk.plot(coords[:, 1], coords[:, 0], color='cyan', marker='x', linestyle='None', markersize=6)

# plt.sca(axk)
# figk.sca(axk)

# cursoraudio = Cursor(axaudio, horizOn = True, vertOn = True, useblit = True, color = 'green', linewidth = 1, zorder = 10)

cursoraudio = MultiCursor(figk.canvas, (axk[0], axk[1], axk[2], axk[3]), useblit = True, horizOn = True, vertOn = True, color = 'red', linewidth = 1)

figk.canvas.draw_idle()

display(figk.canvas)

%reset array

In [ ]:
figf.clear()
axf.clear()

del figf, axf

figf, axf = plt.subplots(num = 4, figsize = (16.7, 6.2))
figf.canvas.header_visible = False

axf.plot(audiokymotime[peakloc[1:]], fsaudio / np.diff(peakloc), 'ro-')

# axf.plot(audiokymotime, audiokymo, 'g.-')

axf.grid(visible = True, which = 'both')

axf.set_xlabel(f'time [frames wrt {fps} fps]')
axf.set_ylabel('local frequency [Hz]')

figf.canvas.draw_idle()

display(figf.canvas)

In [ ]:
from skimage import feature, filters, morphology

In [ ]:
morphology.black_tophat()

In [ ]:
feature.peak_local_max()

In [ ]:
coords = feature.corner_peaks(feature.corner_harris(kymo.T, sigma = 0.5), min_distance = 20)

In [ ]:
from skimage import filters

In [ ]:
elevation_map = filters.sobel(kymo.T)

In [ ]:
del fige, axe

fige, axe = plt.subplots(num = 3, figsize = (4, 3))

axe.imshow(elevation_map, cmap = plt.cm.gray, aspect = 'auto')

display(fige.canvas)

In [ ]:
def find_peaks_fast(xin, neighbors = 1):
    """
    find_peaks_fast(xin, neighbors = 1)
    
    neighbors: search for maxima outside of the immediate neighborhood of total size 2*neighbors
    i.e. minimum period corresponds to neighbors * samplingrate
    
    return [peakloc, mad]
    """
    
    if isinstance(xin, list):
        x = np.array(xin)
    else:
        x = xin

    Nx = x.size

    if neighbors == 0:
        neighbors = 1

    # always an odd number larger or equal to 3
    Nsearch = 2 * neighbors + 1

    if Nsearch > Nx:
        raise ValueError("searchwindow Nsearch is larger than number of samples: reduce neighbors")

    # 1. find maximum
    # 2. is maximum at center? 
    # if no, center searchwindow at maximum, 
    #         search max
    #         test again for center
    # if yes: move search window so that it just does not include the found maximum
    # make sure that the search window does not go backwards and gets stuck on the first-found maximum

    def MAD(samples):
        """
        calculate the median absolute deviation: median(abs(x - median(x)))
        """
        return np.median(np.abs(samples - np.median(samples)))

    def borders(center, neighbors = neighbors):
        left = center - neighbors
        right = center + neighbors + 1
        return left, right

    center = neighbors
    left, right = borders(center)

    searchdata = x[left:right]
    searchdata_zeros = np.zeros_like(searchdata)

    peakloc = []
    mad = []

    # initial step to find first maximum
    while True:
        if left < 0:
            if right < 0:
                print("left: ", left)
                print("center: ", center)
                print("right: ", right)
            searchdata = searchdata_zeros[:]
            searchdata[-left : ] = x[0:right]
            minx = np.min( x[0:right] )
            searchdata[ : -left] = minx
            del minx
        else:
            searchdata = x[left:right]

        maxind = np.argmax(searchdata)
        # maximum at center?
        if maxind == neighbors:
            # found one local maximum: stop this loop
            peakloc.append(center)
            mad.append(MAD(searchdata))
            break
        else:
            # re-center search window and test again
            # work in absolut sample indices, not just relative search window indices
            center = left + maxind
            left, right = borders(center)

    # next step to find more maxima to the RIGHT of the first-found maximum
    #
    # move search window and repeat local search (are we doing mean-shift)
    center = right
    left, right = borders(center)

    # to move forward: discard maximum found at border which is just next to the previously found maximum

    while True:
        if center >= Nx:
            break

        if right > Nx:
            searchdata = searchdata_zeros[:]
            searchdata[0 : Nx - left] = x[left:Nx]
            minx = np.min( x[left:Nx] )
            searchdata[Nx - left : ] = minx
            del minx
        else:
            searchdata = x[left:right]

        maxind = np.argmax(searchdata)

        # 1. is the maximum in the center: done
        # 2. is the maximum to the right of the center: move the center there
        # 3. none of the above: move the entire search window over: center at previous right side

        if maxind == neighbors:
            peakloc.append(center)
            # print "peakloc: ", peakloc
            mad.append(MAD(searchdata))

            # done: move the search window over: center is at previous right side
            center = right
            left, right = borders(center)
            continue
        # this case is not necessary because the next case controls the behavior
        # elif maxind == 0:
        #     # discard border maxima: move forward by half the search window
        #     center = right
        #     left, right = borders(center)

        if maxind > neighbors:
            # re-center search window and test again
            center = left + maxind
            left, right = borders(center)
        else:
            # maxind < neighbors: no candidates for maxima here, move on
            center += 1
            left, right = borders(center)

    return [peakloc, mad]

In [ ]:
def find_peaks(xin, neighbors):
    """
    find_peaks(xin, neighbors)
    """
    # peaks = []

    if isinstance(xin, list):
        x = np.array(xin)
    else:
        x = xin

    nxin = x.size

    # can we know this better based on the max frequency to be detected???
    Ntmp = 10000

    tmp = np.zeros(Ntmp, dtype = np.int64)
    pcount = 0

    assert 2 * neighbors + 1 <= nxin

    if nxin == 1:
        # peaks.append(0)
        tmp[pcount] = 0
        peaks = tmp[pcount]
        return peaks
    elif nxin == 2:
        if x[0] > x[1]:
            # peaks.append(0)
            tmp[pcount] = 0
        else:
            # peaks.append(1)
            tmp[pcount] = 1
        
        peaks = tmp[pcount]
        return peaks

    # pad at the ends of the time series
    minx = x.min()

    padstart = minx * np.ones(neighbors)
    x = np.append(padstart, x)
    
    padend = minx * np.ones(neighbors - 1)
    x = np.append(x, padend)
    
    nx = x.size

    # see mplab.py and the indexing in the spectral_helper function for
    # the sliced time series for the spectrogram
    Nwindow = 2 * neighbors + 1
    step = 1
    ind = np.arange(0, nx - Nwindow + 1, step)
    n = ind.size
    
    # check whether the central point is the maximum
    for i in np.arange(n):
        xsel = x[ind[i]: ind[i] + Nwindow]
        
        # don't need to check in Nwindow after a local maximum was found
        # not implemented yet!
        delta_i = xsel.argmax()

        if xsel.argmax() == neighbors:
            tmp[pcount] = ind[i] + neighbors
            pcount += 1
            delta_i = 1

        new_i = ind[i] + delta_i

        # enlarge the array that holds the extrema location
        if pcount == tmp.size:
            tmp = np.resize(tmp, (pcount + Ntmp,))
            tmp[pcount:] = 0

    # subtract the amount of padded data points at the start
    peaks = tmp[:pcount] - neighbors
    
    if peaks[-1] == (nxin - 1):
        peaks = np.delete(peaks, -1)
        
    if peaks[0] == 0:
        peaks = np.delete(peaks, 0)

    # print "peaks[-1] = ", peaks[-1]
    # print "nxin = ", nxin
    
    return peaks

#######################################

if __name__ == '__main__':
    Nperiods = 20
    period = 3

    x = np.arange(Nperiods * period) % period

    peaks, _ = find_peaks_fast(x, period - 1)

    assert len(peaks) == Nperiods
    pass

# %timeit find_peaks(x, 2)
# 1000 loops, best of 3: 233 us per loop

# %timeit find_peaks_fast(x, 2)
# 10000 loops, best of 3: 38.1 us per loop